In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os
import glob
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

## SVM（先頭30フレームの平均）

In [2]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_30_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.996101  0.621411  0.310666  0.479798  0.311649  0.183596   
answer2     0.997114  0.620886  0.345670  0.512574  0.275988  0.183498   
answer3     0.995380  0.642572  0.413375  0.471471  0.396075  0.158026   
answer4     0.994671  0.690717  0.408082  0.437803  0.420779  0.149670   
answer5     0.994515  0.681902  0.402748  0.464709  0.434606  0.151975   
answer6     0.995428  0.686710  0.527085  0.380380  0.428165  0.162578   
answer7     0.994965  0.627859  0.411296  0.451731  0.489222  0.190342   
answer8     0.994378  0.681677  0.412118  0.554543  0.326517  0.161899   
answer9     0.996479  0.589496  0.367464  0.484712  0.404582  0.206949   
answer10    0.994109  0.695505  0.467914  0.500212  0.497803  0.179128   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.999145  0.552042  0.246321  0.512950  0.247135  0.177600   
answer2     0.986787  0.388134  0.280582  0.290068  0.336429  0.105564   
answer3     0.999173  0.816289  0.473348  0.459009  0.436735  0.081152   
answer4     0.999177  0.790233  0.461463  0.568614  0.378147  0.089520   
answer5     0.999433  0.828134  0.706821  0.237619  0.425510  0.076750   
answer6     0.999672  0.732152  0.344032  0.690645  0.424139  0.081590   
answer7     0.999563  0.807059  0.404164  0.555602  0.394952  0.065848   
answer8     0.999774  0.812318  0.600394  0.381842  0.356839  0.121444   
answer9     0.999602  0.693477  0.199256  0.819582  0.394998  0.074083   
answer10    0.999662  0.822137  0.626957  0.357374  0.400909  0.123460   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.996463  0.390384  0.179116  0.701085  0.297411  0.135393   
answer2     0.998654  0.373085  0.278681  0.714336  0.391086  0.074561   
answer3     0.998215  0.515694  0.420780  0.577842  0.386636  0.124716   
answer4     0.997661  0.602553  0.450953  0.471292  0.348750  0.081385   
answer5     0.997693  0.427534  0.344818  0.510675  0.379461  0.121074   
answer6     0.997671  0.389221  0.311491  0.498827  0.382046  0.124102   
answer7     0.998256  0.460744  0.313374  0.555281  0.311047  0.093776   
answer8     0.997640  0.429439  0.309877  0.536239  0.292645  0.098506   
answer9     0.997832  0.414240  0.328803  0.560852  0.330107  0.081058   
answer10    0.997874  0.434963  0.338820  0.475162  0.333468  0.105434   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [22]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
クラス1の確率: [0.27561144 0.29670685 0.14409688 0.13185822 0.28555841 0.18740037
 0.21645398 0.5        0.59530996 0.24285549]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 60.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 1, 0, 0, 1]
クラス1の確率: [0.21715343 0.54396208 0.13311194 0.75910695 0.18541193 0.15016449
 0.75202493 0.06316809 0.09757506 0.51977634]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.09224786 0.13715641 0.10977778 0.09457584 0.26644612 0.12455545
 0.1650121  0.11873519 0.0937331  0.06314713]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.20344036 0

In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.39498064 0.36518169 0.21788036 0.20133024 0.36344154 0.24627263
 0.31912246 0.48700598 0.5        0.37515435]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 60.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 1, 0, 0, 1]
クラス1の確率: [0.33396059 0.93032926 0.26578105 0.73840985 0.18482287 0.30122611
 0.57457549 0.16987212 0.11653733 0.58960026]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.20474977 0.21038669 0.18428488 0.19679836 0.35000905 0.19047331
 0.2307364  0.20921014 0.16266844 0.16466906]

===== テストデータ: ファイル

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.40714147 0.59221867 0.52999241 0.36716985 0.41088144 0.58265374
 0.396125  ]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.23658618 0.3479076  0.26570227 0.16996689 0.30369506 0.21678794
 0.30967565 0.2508085  0.22406526 0.22575291]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID72_smoothed.csv
予測率: 12.50%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 1, 1, 0, 1, 1, 1, 1]
クラス1の確率: [0.69406476 0.65531336 0.62948792 0.46832196 0.69465256 0.73216683
 0.54199899 0.81216681]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID73_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 

## SVM（先頭10フレームの平均）

In [4]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_10_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.996606  0.601540  0.310669  0.446768  0.329936  0.174976   
answer2     0.997146  0.582202  0.349965  0.518763  0.280240  0.176288   
answer3     0.996037  0.586778  0.323077  0.512440  0.306640  0.165145   
answer4     0.994471  0.688244  0.389618  0.471758  0.387922  0.144351   
answer5     0.994645  0.645536  0.329584  0.537709  0.398903  0.168681   
answer6     0.995012  0.646885  0.477473  0.382850  0.379168  0.163435   
answer7     0.995345  0.616554  0.359053  0.489489  0.432771  0.202691   
answer8     0.994680  0.674456  0.370218  0.526292  0.357387  0.164901   
answer9     0.995906  0.629347  0.358147  0.483077  0.408082  0.176283   
answer10    0.993830  0.710357  0.484428  0.420916  0.505945  0.167669   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer2     0.988677  0.524954  0.447298  0.498014  0.360570  0.140186   
answer3     0.992118  0.577262  0.464889  0.457503  0.361048  0.194862   
answer5     0.964830  0.540084  0.450977  0.374830  0.363270  0.265250   
answer8     0.981836  0.589962  0.502958  0.334950  0.376152  0.270962   
answer9     0.993460  0.689793  0.548061  0.522783  0.407470  0.123633   
answer10    0.978699  0.666391  0.545053  0.486370  0.301076  0.152154   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                  
answer2   0.207236  0.101339  0.067102  0.259279  0.521434  0.535891   
answer3   0.245512  0.093931  0.086200  0.225030  0.431761  0.546676   
answer5   0.343269  0.164045  0.126843  0.332394  0.517765  0.540074   
answer8   0.357463  0.280653  0.216102  0.35114

In [24]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.24354873 0.39227158 0.41002573 0.31521952 0.5        0.24136797
 0.4560525  0.51125224 0.52403009 0.26538994]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.19148832 0.63064861 0.35080849 0.36025033 0.19830912 0.20807228
 0.30377848 0.08994569 0.11878422 0.43508711]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.13976274 0.14844538 0.07841199 0.0894928  0.17917228 0.17280168
 0.13938807 0.14784885 0.07196651 0.06365108]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.04706842 

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.34959494 0.32924857 0.39390478 0.24743089 0.42670017 0.205762
 0.38618725 0.44151725 0.41305813 0.27531195]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.26936788 0.95577027 0.30003281 0.51423577 0.19498893 0.20431015
 0.25732535 0.12004499 0.12101839 0.56376306]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.15865354 0.16842385 0.09650572 0.10729189 0.23387808 0.20819926
 0.15586942 0.15233328 0.10510562 0.09991252]

===== テストデータ: ファイル 4/31 =

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.54016485 0.80179667 0.79372657 0.42211874 0.34378479 0.75862457
 0.56521776]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.28591168 0.25090265 0.31813082 0.18158426 0.24822864 0.24877819
 0.2650252  0.24201923 0.27039692 0.20340384]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID72_smoothed.csv
予測率: 0.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
クラス1の確率: [0.67126321 0.68313298 0.68037785 0.5482333  0.61162652 0.76732156
 0.56645482 0.79658649]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID73_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 

## SVM（先頭30フレームの標準偏差）

In [6]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001530  0.019816  0.015263  0.043458  0.039679  0.024692   
answer2     0.000945  0.036016  0.030624  0.016067  0.019143  0.011539   
answer3     0.001572  0.043948  0.069743  0.042619  0.075438  0.013640   
answer4     0.001297  0.023535  0.022897  0.034610  0.033839  0.018583   
answer5     0.001613  0.030034  0.058682  0.063061  0.046598  0.021786   
answer6     0.001492  0.035972  0.063001  0.025672  0.045928  0.013917   
answer7     0.001258  0.014414  0.046258  0.036940  0.063940  0.020970   
answer8     0.001798  0.013718  0.041780  0.050493  0.031532  0.015747   
answer9     0.001176  0.050766  0.018523  0.044329  0.028126  0.027568   
answer10    0.001767  0.019214  0.023940  0.072549  0.034567  0.012300   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer2     0.011610  0.025869  0.039144  0.090949  0.013338  0.049964   
answer3     0.010687  0.013997  0.035309  0.061364  0.011105  0.031083   
answer4     0.022786  0.040700  0.051660  0.089426  0.031215  0.070175   
answer6     0.027694  0.039053  0.051621  0.099852  0.026355  0.032501   
answer7     0.023408  0.030430  0.039979  0.027458  0.014480  0.039590   
answer8     0.018824  0.022150  0.042481  0.027866  0.005841  0.049403   
answer9     0.025565  0.023842  0.059836  0.055688  0.019503  0.055259   
answer10    0.014909  0.028230  0.041145  0.083813  0.018164  0.032775   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                  
answer2   0.053918  0.060399  0.056313  0.027788  0.044563  0.024605   
answer3   0.011023  0.024007  0.042753  0.0

In [26]:

from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.30753726 0.26448501 0.34001028 0.31942499 0.37604727 0.31604811
 0.34046073 0.42393985 0.34582513 0.40299472]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
クラス1の確率: [0.38865412 0.48035079 0.44474815 0.47672877 0.5        0.37615592
 0.32341323 0.27773445 0.3039054  0.47836495]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.17986098 0.29074902 0.29547136 0.26801078 0.3067048  0.28243624
 0.25972485 0.15632325 0.21497308 0.36207032]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.40434266 

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.36237297 0.3167669  0.28691347 0.35097552 0.39971985 0.34933115
 0.34474874 0.38601853 0.38174521 0.41098885]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.43089797 0.44496581 0.4715151  0.57428657 0.5        0.41917657
 0.17699669 0.36685483 0.39582493 0.48913695]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.26505412 0.35689058 0.36354506 0.3446079  0.36559692 0.37942225
 0.31894483 0.23601613 0.31504768 0.38367332]

===== テストデータ: ファイル 4/31 ==

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.28020351 0.43278097 0.37398329 0.32065745 0.37461483 0.35569733
 0.36517782]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.39843897 0.36636883 0.39072962 0.36656573 0.47323906 0.35336339
 0.49052561 0.39597625 0.35589988 0.36766662]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID72_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.40447336 0.43844967 0.51599171 0.44841008 0.450107   0.47622709
 0.40302165 0.45232569]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID73_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0,

## SVM（先頭10フレームの標準偏差）

In [2]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001006  0.009949  0.010251  0.015674  0.021276  0.007454   
answer2     0.001069  0.013489  0.030429  0.011434  0.019826  0.014543   
answer3     0.001248  0.020389  0.027520  0.029420  0.003884  0.013260   
answer4     0.001489  0.015146  0.014760  0.034909  0.011303  0.006671   
answer5     0.001968  0.007569  0.016442  0.058506  0.037077  0.015832   
answer6     0.001509  0.020592  0.051012  0.026717  0.028588  0.010255   
answer7     0.001398  0.016808  0.018877  0.007412  0.020422  0.017243   
answer8     0.001651  0.007443  0.022618  0.010983  0.004949  0.021055   
answer9     0.000932  0.042140  0.010872  0.033027  0.035538  0.006528   
answer10    0.002075  0.021212  0.011367  0.018154  0.023781  0.003617   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer2     0.000125  0.027295  0.014951  0.055959  0.000938  0.024016   
answer3     0.000421  0.021759  0.039955  0.015174  0.003932  0.009044   
answer4     0.000034  0.030689  0.009992  0.046596  0.006658  0.028914   
answer6     0.001302  0.047371  0.026906  0.067178  0.034557  0.054390   
answer7     0.000315  0.011450  0.014419  0.023161  0.030156  0.010754   
answer8     0.000149  0.017209  0.009959  0.008754  0.001887  0.005567   
answer9     0.000189  0.004324  0.019252  0.033411  0.032320  0.008020   
answer10    0.000279  0.005377  0.008937  0.015730  0.010740  0.009821   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                  
answer2   0.011650  0.066919  0.016386  0.011620  0.079268  0.071335   
answer3   0.011809  0.088392  0.020586  0.0

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.000277  0.004820  0.006261  0.020796  0.001755  0.003508   
answer2     0.004617  0.012507  0.019467  0.034434  0.005028  0.006887   
answer3     0.000139  0.005064  0.048782  0.018487  0.002687  0.003400   
answer4     0.000084  0.005261  0.015534  0.017555  0.003390  0.000700   
answer5     0.000071  0.013879  0.021444  0.029228  0.013218  0.001374   
answer6     0.000051  0.007389  0.056186  0.007652  0.007868  0.003877   
answer7     0.000064  0.004559  0.028096  0.015314  0.005686  0.003252   
answer8     0.000022  0.022920  0.015182  0.029995  0.018126  0.019775   
answer9     0.000041  0.021986  0.004019  0.015120  0.013790  0.000952   
answer10    0.000053  0.013600  0.004661  0.056279  0.004777  0.032322   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001600  0.046005  0.027362  0.043430  0.013830  0.027783   
answer2     0.001152  0.059428  0.084419  0.059925  0.011647  0.010453   
answer3     0.000895  0.056196  0.069513  0.033642  0.014289  0.012335   
answer5     0.005583  0.040388  0.018813  0.015786  0.026725  0.016100   
answer6     0.006500  0.017881  0.023929  0.019402  0.008774  0.008952   
answer7     0.004119  0.043871  0.027193  0.063105  0.037051  0.027733   
answer8     0.003280  0.040392  0.087241  0.070512  0.028752  0.049861   
answer9     0.001072  0.022310  0.037779  0.050148  0.012653  0.039939   
answer10    0.004875  0.013354  0.037033  0.066154  0.011131  0.036202   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                  
answer1   0.022669  0.047259  0.017125  0

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.000141  0.017773  0.006525  0.011747  0.008603  0.003432   
answer2     0.000307  0.004447  0.006031  0.006765  0.003813  0.004455   
answer3     0.000104  0.004053  0.010283  0.027268  0.011809  0.002667   
answer4     0.000222  0.008502  0.011863  0.021073  0.014376  0.005187   
answer6     0.000171  0.004216  0.017386  0.025858  0.014623  0.002297   
answer7     0.000257  0.028716  0.029535  0.038061  0.013011  0.002852   
answer8     0.000320  0.003174  0.010730  0.015947  0.018471  0.004936   
answer9     0.000177  0.016266  0.017108  0.019675  0.015027  0.008574   
answer10    0.000207  0.006105  0.006279  0.022181  0.007933  0.004775   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                  
answer1   0.001029  0.022298  0.002711  0

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001812  0.041888  0.026062  0.013870  0.007326  0.008585   
answer2     0.010123  0.053507  0.030017  0.060622  0.022691  0.003801   
answer3     0.000609  0.018117  0.009134  0.026551  0.006641  0.007801   
answer4     0.000770  0.034607  0.038773  0.031512  0.013551  0.003466   
answer6     0.002604  0.025423  0.020062  0.015696  0.012903  0.012777   
answer7     0.001357  0.020039  0.028594  0.030234  0.009262  0.005137   
answer8     0.001116  0.012800  0.019839  0.024059  0.016601  0.013010   
answer9     0.001782  0.026626  0.019013  0.048522  0.011110  0.028319   
answer10    0.000678  0.022065  0.017678  0.020418  0.007088  0.018536   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                                                  
answer1   0.020380  0.017503  0.028558  0

In [3]:
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.26908712 0.25598377 0.22507412 0.32235102 0.40521875 0.30949287
 0.39890836 0.58129256 0.18679271 0.21732885]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 0, 0, 0, 1, 0, 0, 0]
クラス1の確率: [0.22806079 0.57095905 0.40349097 0.28626559 0.4124687  0.33244913
 0.62881483 0.20972052 0.29138699 0.36042085]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.34830386 0.14602674 0.14695525 0.16122471 0.22572486 0.13082338
 0.24790399 0.1805416  0.13668307 0.41904836]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.18927298 0

In [4]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.28854517 0.33184277 0.17934357 0.35942721 0.40551863 0.40439208
 0.35008113 0.46716526 0.33849931 0.30375262]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.26459431 0.51327398 0.43504857 0.28708485 0.53642604 0.39343957
 0.44316771 0.33106353 0.33223252 0.3601616 ]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.27521228 0.22674009 0.21799656 0.19599474 0.28581898 0.20924127
 0.2675203  0.20696218 0.22543421 0.34399106]

===== テストデータ: ファイル 4/31 ==

最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 28.57%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [1, 0, 0, 1, 0, 0, 0]
クラス1の確率: [0.80006328 0.58025916 0.43625122 0.8316395  0.45163394 0.61196358
 0.44661616]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 1, 1, 0, 0]
クラス1の確率: [0.42671741 0.20365916 0.36834484 0.31177595 0.54856942 0.56940914
 0.66919025 0.66944946 0.22859559 0.39720712]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID72_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.31881949 0.5186068  0.42024223 0.41087902 0.50598663 0.37958672
 0.2860758  0.29147207]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID73_smoothed.csv
予測率: 88.89%
正解ラベル: [0, 0, 

## SVM（先頭30フレームの差分）

In [27]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 結果をリストに保存
    result_list.append((file_name, diff_values_per_label))

    # Yを追加
    diff_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
        label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     answer1    0.002523 -0.001195  0.014545  0.005055  0.002442  0.002156   
2     answer1   -0.000777 -0.000953  0.005555 -0.014980  0.017999 -0.007861   
3     answer1   -0.000558 -0.015196 -0.016889  0.003133 -0.018193 -0.002822   
4     answer1    0.000219  0.006084 -0.002855 -0.008630 -0.000543  0.006542   
5     answer1    0.000912 -0.002755 -0.010911  0.001786 -0.009210  0.003166   
..        ...         ...       ...       ...       ...       ...       ...   
295  answer10    0.004307  0.001851 -0.008759  0.021084 -0.012385 -0.002022   
296  answer10   -0.000085  0.008260  0.028112 -0.026262 -0.018536 -0.004130   
297  answer10   -0.003561  0.014570  0.006747 -0.037738 -0.005984 -0.002483   
298  answer10    0.001567  0.006688  0.013547  0.007463 -0.011259  0.004897   
299  answer10    0.001315  0.003890  0.008731 -0.012653 -0.004096 -0.002768   

         AU

In [28]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    #print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    #print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 97.24%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtere

ファイル名: filtered_ID40_smoothed.csv
予測率: 92.80%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 12/31 =====
ファイル名: filtered_ID41_smoothed.csv
予測率: 87.50%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

ファイル名: filtered_ID54_smoothed.csv
予測率: 29.24%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 23/31 =====
ファイル名: filtered_ID56_smoothed.csv
予測率: 19.23%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## SVM（先頭30フレームの差分の標準偏差）

In [8]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 差分の標準偏差を計算
    std_values_per_label = (
        diff_values_per_label.groupby("label").std(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.001462  0.009580  0.012407  0.016297  0.011874  0.007615   
answer2     0.001210  0.012272  0.015193  0.015429  0.009401  0.005542   
answer3     0.001548  0.010425  0.014770  0.011224  0.011701  0.004530   
answer4     0.001596  0.011475  0.010987  0.016799  0.017746  0.005220   
answer5     0.002208  0.008542  0.013593  0.017499  0.017769  0.006108   
answer6     0.001575  0.009464  0.016398  0.014048  0.016414  0.005451   
answer7     0.001426  0.008086  0.011682  0.019850  0.022139  0.005849   
answer8     0.001579  0.008420  0.013003  0.012297  0.006954  0.006512   
answer9     0.001348  0.013357  0.014014  0.018175  0.010446  0.004908   
answer10    0.002133  0.009996  0.012292  0.017410  0.010149  0.003508   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1          NaN       NaN       NaN       NaN       NaN       NaN   
answer2     0.001489  0.007765  0.012411  0.021777  0.004289  0.003467   
answer3     0.000420  0.008880  0.010858  0.016399  0.003883  0.002913   
answer4     0.000628  0.011773  0.011558  0.020367  0.006988  0.009850   
answer5          NaN       NaN       NaN       NaN       NaN       NaN   
answer6     0.019767  0.012390  0.025108  0.034852  0.005774  0.015906   
answer7     0.000108  0.009293  0.012002  0.012152  0.015235  0.002877   
answer8     0.004102  0.008301  0.019756  0.021228  0.005763  0.013388   
answer9     0.000066  0.006051  0.006710  0.012970  0.001475  0.003877   
answer10    0.000099  0.009661  0.010732  0.015345  0.009272  0.003626   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [30]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    #print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34_smoothed.csv
予測率: 12.50%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 1, 0, 0, 0, 0]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.30284097 0.2989328  0.18937846 0.21401034 0.26291118 0.25353223
 0.29322957 0.272002   0.27837243 0.23959097]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.28658009 0.6628192  0.24106414 0.61273791 0.33965188 0.23823171
 0.34526899 0.20680256 0.2245616  0.3438929 ]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.15465869 0.13553165 0.1008484  0.13827089 0.2077778  0.13092195
 0.18575781 0.11315957 0.10341657 0.18906762]

===== テストデータ: ファイル 4/31 ===

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.81690934 0.64727789 0.74460337 0.73651171 0.83136562 0.70546722
 0.80789111]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.38414334 0.43386604 0.39846465 0.27078691 0.37635941 0.38376121
 0.47826217 0.44426285 0.23304867 0.36667422]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID72_smoothed.csv
予測率: 62.50%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 1, 0, 0, 1, 0, 0]
クラス1の確率: [0.39839776 0.6194782  0.74397527 0.381337   0.47653296 0.56253162
 0.29132863 0.32195537]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID73_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 

## SVM（先頭30フレームの最大値）

In [10]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに30フレームの最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.998552  0.659222  0.342934  0.560316  0.386278  0.225416   
answer2     0.998392  0.687580  0.381302  0.549067  0.308328  0.201725   
answer3     0.998105  0.697356  0.488168  0.556769  0.494694  0.183472   
answer4     0.996902  0.730949  0.459202  0.527441  0.481439  0.189705   
answer5     0.997551  0.717898  0.484356  0.631736  0.521545  0.198245   
answer6     0.997832  0.741541  0.631410  0.431417  0.504093  0.194596   
answer7     0.997573  0.657243  0.472553  0.499724  0.594109  0.225480   
answer8     0.997223  0.704030  0.483926  0.640750  0.367383  0.199509   
answer9     0.998192  0.686589  0.426857  0.570879  0.444240  0.246305   
answer10    0.996892  0.746445  0.505533  0.631177  0.537849  0.197410   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.997486  0.565893  0.530230  0.303754  0.413368  0.234803   
answer2     0.996569  0.671848  0.561891  0.454573  0.475025  0.239175   
answer3     0.998651  0.684315  0.566235  0.407668  0.464026  0.207943   
answer4          NaN       NaN       NaN       NaN       NaN       NaN   
answer5     0.996193  0.603913  0.447637  0.472053  0.490529  0.228341   
answer6          NaN       NaN       NaN       NaN       NaN       NaN   
answer7     0.993392  0.665464  0.587689  0.383696  0.466492  0.264016   
answer8          NaN       NaN       NaN       NaN       NaN       NaN   
answer9     0.996393  0.687083  0.563349  0.404818  0.403813  0.370473   
answer10    0.994718  0.650643  0.518384  0.354154  0.389988  0.317829   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [32]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    #print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34_smoothed.csv
予測率: 50.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [1, 1, 1, 1, 0, 0, 0, 0]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35_smoothed.csv
予測率: 88.89%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36_smoothed.csv
予測率: 50.00%
正解ラベル: [0, 0, 0, 0, 0, 

In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.5        0.48165272 0.24816931 0.25273067 0.44555314 0.28793633
 0.24047978 0.53101453 0.32128556 0.39850491]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.42152223 0.70295575 0.39642006 0.64616732 0.35257581 0.34885491
 0.40005494 0.35912508 0.27799123 0.54199948]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.06304743 0.07780776 0.03114193 0.18470418 0.18205517 0.03401356
 0.06010966 0.02982381 0.05021348 0.00688145]

===== テストデータ: ファイル 4/3

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3337512  0.52757013 0.42742275 0.28310523 0.35682551 0.4731734
 0.34801296]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.25656365 0.24643012 0.30875932 0.26914896 0.35005601 0.27618152
 0.34339032 0.30642132 0.28692132 0.28664562]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID72_smoothed.csv
予測率: 0.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
クラス1の確率: [0.70772039 0.48392793 0.80216368 0.42531848 0.77967798 0.5624853
 0.61921705 0.66920895]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID73_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 

## SVM（先頭10フレームの最大値）

In [12]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに10フレームの最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.998552  0.614411  0.325737  0.466249  0.362126  0.192537   
answer2     0.998392  0.611207  0.381302  0.533734  0.308328  0.201725   
answer3     0.998105  0.623145  0.365551  0.556769  0.311882  0.183472   
answer4     0.996902  0.713736  0.410865  0.527441  0.405043  0.151519   
answer5     0.996959  0.660467  0.352912  0.631736  0.443868  0.198245   
answer6     0.997096  0.684254  0.561657  0.431417  0.409441  0.184528   
answer7     0.997573  0.639601  0.378727  0.499724  0.471975  0.225345   
answer8     0.997107  0.685622  0.416294  0.541601  0.367383  0.199509   
answer9     0.996756  0.686589  0.374178  0.536184  0.444240  0.188060   
answer10    0.996236  0.746445  0.505533  0.444868  0.537849  0.172488   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.998608  0.584214  0.487908  0.507573  0.415887  0.300063   
answer2     0.998557  0.621833  0.451983  0.667127  0.423226  0.227818   
answer3     0.998383  0.687069  0.543647  0.552579  0.363243  0.243766   
answer4          NaN       NaN       NaN       NaN       NaN       NaN   
answer5     0.997662  0.520747  0.366398  0.541289  0.386945  0.391653   
answer6     0.995698  0.607228  0.431603  0.591542  0.341358  0.250351   
answer7     0.998125  0.539311  0.296144  0.665015  0.389080  0.302609   
answer8     0.997875  0.681109  0.539304  0.562647  0.411455  0.332636   
answer9     0.996977  0.642100  0.567037  0.587109  0.405672  0.403543   
answer10    0.992794  0.676794  0.630556  0.517442  0.364702  0.345626   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [34]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    #print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    #print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34_smoothed.csv
予測率: 50.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35_smoothed.csv
予測率: 77.78%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36_smoothed.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.45150333 0.5        0.44351168 0.35881407 0.60079849 0.24627179
 0.42931542 0.61168207 0.52488957 0.30318048]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.25891748 0.94689565 0.28955551 0.5        0.23631826 0.19832551
 0.2849652  0.16325907 0.11472739 0.51745717]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.16199893 0.11500837 0.08028133 0.09215628 0.14283792 0.118156
 0.14640377 0.13513313 0.09292123 0.08549741]

===== テストデータ: ファイル 4/31 ====

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.69298954 0.80974509 0.70887188 0.40470838 0.38861385 0.80305464
 0.6366906 ]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.24106942 0.22927934 0.27371837 0.20692093 0.27386521 0.25869543
 0.29991182 0.27470807 0.27623442 0.22189159]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID72_smoothed.csv
予測率: 25.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 1, 1, 0, 1, 1, 0, 1]
クラス1の確率: [0.55656355 0.87132327 0.72537562 0.46112905 0.64595582 0.68632679
 0.36089691 0.72355846]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID73_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0,

## SVM（先頭30フレームの最大値と最小値の差分）

In [14]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_30_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.006316  0.075649  0.063028  0.145261  0.127436  0.088201   
answer2     0.003334  0.118679  0.108644  0.066483  0.071918  0.044361   
answer3     0.005948  0.133020  0.202349  0.147331  0.195239  0.046483   
answer4     0.004306  0.070528  0.087552  0.145972  0.120498  0.066656   
answer5     0.006226  0.080696  0.175598  0.229915  0.178028  0.080477   
answer6     0.005584  0.119441  0.215140  0.099514  0.172244  0.046470   
answer7     0.005535  0.066280  0.144592  0.132392  0.192303  0.063128   
answer8     0.006327  0.042552  0.142992  0.147651  0.081356  0.069137   
answer9     0.004578  0.170315  0.085489  0.146519  0.095403  0.078796   
answer10    0.006379  0.077472  0.080922  0.231260  0.119278  0.035864   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.012158  0.152011  0.190406  0.233182  0.091208  0.151257   
answer2     0.088131  0.097610  0.119647  0.226473  0.089968  0.164029   
answer3     0.048399  0.123253  0.196165  0.281300  0.130782  0.215014   
answer4     0.046821  0.092207  0.106581  0.119497  0.213168  0.157697   
answer5     0.075998  0.171632  0.224630  0.115530  0.214810  0.211177   
answer6     0.074967  0.221238  0.273240  0.075341  0.176055  0.275398   
answer7     0.090318  0.178080  0.204388  0.224705  0.125676  0.275451   
answer8     0.078129  0.273097  0.346222  0.340316  0.294983  0.278059   
answer9     0.067475  0.044990  0.108070  0.161256  0.070375  0.082381   
answer10    0.087835  0.098443  0.098059  0.133704  0.221643  0.155157   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [36]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    #print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    #print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36_smoothed.csv
予測率: 60.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38_smoothed.csv
予測率: 80.00%
正解ラベル: [0,

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.26138821 0.27157388 0.30232438 0.31590097 0.35238884 0.2887413
 0.28210131 0.36970124 0.38099228 0.37261882]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.39068676 0.44835436 0.43720653 0.59678555 0.47775562 0.39819206
 0.27697957 0.3171867  0.36136669 0.51313878]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.24933484 0.3417789  0.31490194 0.32060339 0.33478739 0.31686288
 0.29492584 0.22723781 0.29178548 0.34149424]

===== テストデータ: ファイル 4/31 ===

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4048409  0.56522613 0.22129499 0.41651166 0.33532791 0.39084325
 0.37695227]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.39709537 0.38123053 0.41869773 0.34844732 0.41925207 0.38393339
 0.47945445 0.39353872 0.33491657 0.35755769]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID72_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.37766091 0.46606217 0.58688174 0.44578235 0.44679131 0.46581045
 0.34925359 0.41997643]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID73_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0,

## SVM（先頭10フレームの最大値と最小値の差分）

In [16]:
# 処理するフォルダのパス
folder_path = './output_moving_filtered_data'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `answer1`から`answer10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^answer(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"answer{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_10_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30_smoothed.csv:
          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.003534  0.030838  0.030655  0.051194  0.063530  0.027755   
answer2     0.003334  0.042307  0.099821  0.030257  0.053053  0.044361   
answer3     0.004746  0.058809  0.079733  0.077824  0.012427  0.036891   
answer4     0.004306  0.041482  0.039216  0.105106  0.044102  0.017800   
answer5     0.005634  0.023264  0.044154  0.173628  0.100351  0.047550   
answer6     0.004848  0.062154  0.145386  0.077470  0.077592  0.030245   
answer7     0.004615  0.048639  0.050766  0.021128  0.062392  0.047385   
answer8     0.004237  0.023793  0.075360  0.030263  0.015480  0.060184   
answer9     0.003143  0.114966  0.032810  0.093566  0.095403  0.020552   
answer10    0.005723  0.067025  0.041729  0.044950  0.074137  0.010942   

              AU09      AU10      AU12      AU14      AU15      AU17  \

          Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                    
answer1     0.013018  0.027696  0.092070  0.079292  0.035760  0.015457   
answer2     0.019380  0.057441  0.036748  0.094037  0.037180  0.047416   
answer3     0.027094  0.067018  0.075149  0.049722  0.036728  0.044337   
answer4          NaN       NaN       NaN       NaN       NaN       NaN   
answer5     0.045196  0.114671  0.093770  0.091466  0.156165  0.024004   
answer6          NaN       NaN       NaN       NaN       NaN       NaN   
answer7     0.049929  0.056879  0.052926  0.110896  0.079513  0.022700   
answer8          NaN       NaN       NaN       NaN       NaN       NaN   
answer9     0.021083  0.036601  0.052889  0.098063  0.025266  0.124380   
answer10    0.022920  0.038005  0.124970  0.060792  0.038735  0.048162   

              AU09      AU10      AU12      AU14      AU15      AU17  \
label                                  

In [38]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    #print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    #print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31_smoothed.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34_smoothed.csv
予測率: 25.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38_smoothed.csv
予測率: 60.00%
正解ラベル: [0

In [17]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID30_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
クラス1の確率: [0.30929345 0.34458123 0.17376003 0.36328562 0.51786849 0.4250463
 0.28574862 0.47264145 0.33018887 0.29361892]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID31_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.26913249 0.54254829 0.40278608 0.29526241 0.56124736 0.38187834
 0.44168847 0.35309245 0.32035403 0.37605176]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32_smoothed.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.26569509 0.20971091 0.20339733 0.17846055 0.27048863 0.19364317
 0.27502225 0.19419167 0.1967296  0.36955877]

===== テストデータ: ファイル 4/31 ===

最適パラメータ: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID70_smoothed.csv
予測率: 14.29%
正解ラベル: [1, 1, 1, 1, 1, 1, 1]
予測ラベル: [1, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.67012752 0.47903936 0.42633882 0.59205407 0.41509145 0.50760402
 0.42658845]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID71_smoothed.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
クラス1の確率: [0.41220958 0.2094668  0.40162749 0.26894511 0.57247954 0.52088201
 0.72905583 0.6411775  0.20658283 0.37517897]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID72_smoothed.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.31988805 0.5908988  0.44557826 0.42232955 0.52151123 0.33558381
 0.27441775 0.29197556]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID73_smoothed.csv
予測率: 88.89%
正解ラベル: [0,